# 下面提供bert模型解决关键词提取的思路  
当我们想从特定文档中了解关键信息时，通常会转向关键词提取（keyword extraction）。关键词提取（keyword extraction）是提取与输入文本最相关的词汇、短语的自动化过程。  
目前已经有很多的集成工具库类似KeyBERT，已经能够达到非常好的效果，但是在这里我们选择使用BERT创建自己的关键词提取模型，来帮助大家更好的完成文本关键词提取的整个流程

## 导入前置依赖

In [1]:
# 导入pandas用于读取表格数据
import pandas as pd

# 导入BOW（词袋模型），可以选择将CountVectorizer替换为TfidfVectorizer（TF-IDF（词频-逆文档频率）），注意上下文要同时修改，亲测后者效果更佳
from sklearn.feature_extraction.text import TfidfVectorizer
# 导入Bert模型
from sentence_transformers import SentenceTransformer

# 导入计算相似度前置库，为了计算候选者和文档之间的相似度，我们将使用向量之间的余弦相似度，因为它在高维度下表现得相当好。
from sklearn.metrics.pairwise import cosine_similarity

# 过滤警告消息
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

texts=["dog cat fish","dog cat cat","fish bird", 'bird'] # “dog cat fish” 为输入列表元素,即代表一个文章的字符串
cv = TfidfVectorizer()#创建词袋数据结构
cv_fit=cv.fit_transform(texts)
#上述代码等价于下面两行
#cv.fit(texts)
#cv_fit=cv.transform(texts)

print(cv.get_feature_names())    #['bird', 'cat', 'dog', 'fish'] 列表形式呈现文章生成的词典

print(cv.vocabulary_	)              # {‘dog’:2,'cat':1,'fish':3,'bird':0} 字典形式呈现，key：词，value:词频

print(cv_fit)      # 第n个列表元素，**词典中索引为n的元素**， 词频


['bird', 'cat', 'dog', 'fish']
{'dog': 2, 'cat': 1, 'fish': 3, 'bird': 0}
  (0, 3)	0.5773502691896257
  (0, 1)	0.5773502691896257
  (0, 2)	0.5773502691896257
  (1, 1)	0.8944271909999159
  (1, 2)	0.4472135954999579
  (2, 0)	0.7071067811865475
  (2, 3)	0.7071067811865475
  (3, 0)	1.0


## 读取数据集并处理

In [ ]:
# 读取数据集
test = pd.read_csv('./基于论文摘要的文本分类与关键词抽取挑战赛公开数据/test.csv')
test['title'] = test['title'].fillna('')
test['abstract'] = test['abstract'].fillna('')


test['text'] = test['title'].fillna('') + ' ' +test['abstract'].fillna('')

In [2]:

# 定义停用词，去掉出现较多，但对文章不关键的词语
stops =[i.strip() for i in open(r'stop.txt',encoding='utf-8').readlines()] 


这里我们使用distiluse-base-multilingual-cased，因为它在相似性任务中表现出了很好的性能，这也是我们对关键词/关键短语提取的目标!
由于transformer模型有token长度限制，所以在输入大型文档时，你可能会遇到一些错误。在这种情况下，您可以考虑将您的文档分割成几个小的段落，并对其产生的向量进行平均池化（mean pooling ，要取平均值）。

In [ ]:
model = SentenceTransformer(r'xlm-r-distilroberta-base-paraphrase-v1')

In [4]:
test_words = []
for row in test.iterrows():
    # 读取第每一行数据的标题与摘要并提取关键词
    
    n_gram_range = (2,2)
    # 这里我们使用TF-IDF算法来获取候选关键词 
    count = TfidfVectorizer(ngram_range=n_gram_range, stop_words=stops).fit([row[1].text])
    candidates = count.get_feature_names_out()
    # 将文本标题以及候选关键词/关键短语转换为数值型数据（numerical data）。我们使用BERT来实现这一目的
    title_embedding = model.encode([row[1].title])
    
    candidate_embeddings = model.encode(candidates)
    
    # 通过修改这个参数来更改关键词数量
    top_n = 15
    # 利用文章标题进一步提取关键词
    distances = cosine_similarity(title_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    
    if len( keywords) == 0:
         keywords = ['A', 'B']
    test_words.append('; '.join( keywords))
    

c:\Users\lima\anaconda3\envs\pytorch\lib\site-packages\sklearn\feature_extraction\text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  % sorted(inconsistent)
c:\Users\lima\anaconda3\envs\pytorch\lib\site-packages\sklearn\feature_extraction\text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  % sorted(inconsistent)
c:\Users\lima\anaconda3\envs\pytorch\lib\site-packages\sklearn\feature_extraction\text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words gener

得到结果，最后输出为要求格式

In [5]:
test['Keywords'] = test_words
test[['uuid', 'Keywords']].to_csv('submit_task2.csv', index=None)